# Intro to `pandas`

We'll explore the Pandas package for simple data handling tasks using geoscience data examples. 

### Data management for machine learning
- DataFrames: A new way to look at well logs.
- DataFrames vs arrays.




## Basic Pandas

Introduces the concept of a `DataFrame` in Python. If you're familiar with R, it's pretty much the same idea! Useful cheat sheet [here](https://www.datacamp.com/community/blog/pandas-cheat-sheet-python#gs.59HV6BY)

The main purpose of Pandas is to allow easy manipulation of data in tabular form. Perhaps the most important idea that makes Pandas great for data science, is that it will always preserve **alignment** between data and labels.

In [ ]:
import pandas as pd
import numpy as np

The most common data structure in Pandas is the `DataFrame`. A 2D structure that can hold various types of Python objects indexed by an `index` array (or multiple `index` arrays). Columns are usually labelled as well using strings.

An easy way to think about a `DataFrame` is if you imagine it as an Excel spreadsheet.

Let's define one using a numpy array:

In [ ]:
arr =  [[2.13, 'sandstone'],
        [3.45, 'limestone'],
        [2.45, 'shale']]
arr

Make a `DataFrame` from `arr`

In [ ]:
df = pd.DataFrame(arr, columns=['velocity', 'lithology'])
df

Accessing the data is a bit more complex than in the numpy array cases but for good reasons

In [ ]:
df.loc[df['velocity'] < 3, 'lithology']


## Adding data

Add more data (row wise)

In [ ]:
df.loc[3] = [2.6, 'shale']
df

Add data (column wise) specifying the index locations

In [ ]:
df.loc[0:2, 'one_more_column'] = [6, 7, 8]
df

Add a new column with a "complete" list, array or series

In [ ]:
df['second_new_column'] = ["x", "y", "z", "a"]
df

## Reading a CSV

Pandas also reads files from disk in tabular form ([here](http://pandas.pydata.org/pandas-docs/version/0.20/io.html)'s a list of all the formats that it can read and write). A very common one is CSV, so let's load one!

In [ ]:
df = pd.read_csv("../data/2016_ML_contest_training_data.csv")
df.head()

<div class="alert alert-success">
Create a new column called "ILD" and store in it the value of 10 to the power of the values in column "ILD_log10".

Check the Pandas documentation [here](http://pandas.pydata.org/pandas-docs/version/0.22/api.html#data-manipulations) and look for a way to determine how many different facies are part of the `DataFrame`.
</div>

In [ ]:
df.groupby('Facies').size()

# Inspecting the `DataFrame`

Using the `DataFrame` with well log information loaded before, we can make a summary using the `describe()` method of the `DataFrame` object

In [ ]:
df.describe()

In [ ]:
df = df.dropna()

## Adding more data to the `DataFrame`

We'd like to augment the DataFrame with some new data, based on some of the existing data.

In [ ]:
def calc_phi_rhob(phind, deltaphi):
    """
    Compute phi_RHOB from phi_ND and Delta_phi.
    """
    return 2 * (phind/100) / (1 - deltaphi/100) - deltaphi/100

In [ ]:
def calc_rhob(phi_rhob, rho_matrix=2650.0, rho_fluid=1000.0):
    """
    Returns density porosity log.
    
    Some typical values for rho_matrix:
      Sandstone:  2650 kg/m^3
      Limestone:  2710 kg/m^3
      Dolomite:   2880 kg/m^3
      Anyhydrite: 2980 kg/m^3
      Salt:       2030 kg/m^3

    Some typical values for rho_fluid:
      Fresh water: 1000 kg/m^3
      Salt water:  1100 kg/m^3
      Heavy oil:   1000 kg/m^3
      Light oil:    800 kg/m^3
      LNG:          650 kg/m^3
    
    See wiki.aapg.org/Density-neutron_log_porosity
    """
    return rho_matrix * (1 - phi_rhob) + rho_fluid * phi_rhob

In [ ]:
phi_rhob = calc_phi_rhob(df.PHIND, df.DeltaPHI)
df['RHOB'] = calc_rhob(phi_rhob)

In [ ]:
df.describe()

We can define a Python dictionary to relate facies with the integer label on the `DataFrame`

In [ ]:
facies_dict = {1:'sandstone', 2:'c_siltstone', 3:'f_siltstone', 4:'marine_silt_shale',
               5:'mudstone', 6:'wackestone', 7:'dolomite', 8:'packstone', 9:'bafflestone'}

Let's add a new column with the name version of the facies

In [ ]:
df["s_Facies"] = [facies_dict.get(x, "Unknown") for x in df.Facies]

In [ ]:
df.head()

There is also a `replace` method on DataFrames and Series, and it takes a dictionary for what to replace with what. So we could also achieve the same thing by passing our dictionary to that.

In [ ]:
df["s_Facies"] = df["Facies"].replace(facies_dict)

If the key is not in the dict, then it is not replaced and the original key is used instead:

In [ ]:
df.loc[0:2, 'Facies'] = 99

In [ ]:
df["s_Facies"] = df["Facies"].replace(facies_dict)

In [ ]:
df.head(10)

In [ ]:
df.loc[0:2, 'Facies'] = 3
df["s_Facies"] = df["Facies"].replace(facies_dict)

In [ ]:
df.head(10)

## Visual exploration of the data

We can easily visualize the properties of each facies and how they compare using a `PairPlot`. The library `seaborn` integrates with matplotlib to make these kind of plots easily.

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(df,
             hue="s_Facies",
             vars=['GR','RHOB','PE','ILD_log10'])

We can have a lot of control over all of the elements in the pair-plot by using the `PairGrid` object.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

g = sns.PairGrid(df, hue="s_Facies", vars=['GR','RHOB','PE','ILD_log10'], size=4)

g.map_upper(plt.scatter,**dict(alpha=0.4))  
g.map_lower(plt.scatter,**dict(alpha=0.4))
g.map_diag(plt.hist,**dict(bins=20))  
g.add_legend()

It is very clear that it's hard to separate these facies in feature space. Let's just select a couple of facies and using Pandas, select the rows in the `DataFrame` that contain information about those facies 

In [ ]:
selected = ['f_siltstone', 'bafflestone', 'wackestone']

dfs = pd.concat([df[df.s_Facies == x] for x in selected])

In [ ]:
g = sns.PairGrid(dfs, hue="s_Facies", vars=['GR','RHOB','PE','ILD_log10'], size=4)  
g.map_upper(plt.scatter, alpha=0.4)
g.map_lower(plt.scatter, alpha=0.4)
g.map_diag(plt.hist,**dict(bins=20))  
g.add_legend()

In [ ]:
dfs = dfs.sort_values(['Well Name', 'Depth'])

In [ ]:
dfs.head()

In [ ]:
dfs.to_csv("../data/training_DataFrame_processed.csv")

---

**INTRO TO GEOCOMPUTING STUDENTS STOP HERE**

---

## Exploring data beyond Matplotlib

A few other plotting libraries have emerged with the rise in popularity of data science that make use of JavaScript to add interactivity. Examples:
 - [Altair](https://altair-viz.github.io/index.html)
 - [Holoviews](http://holoviews.org/index.html)

## Altair
To install, activate `geocomp` and type:
- `pip install -U altair vega_datasets notebook vega`


In [ ]:
import altair as alt
alt.renderers.enable('notebook')

In [ ]:
import altair as alt

xscale = alt.Scale(domain=(0, 350.0))
yscale = alt.Scale(domain=(0, 35))

area_args = {'opacity': .5, 'interpolate': 'step'}
blank_axis = alt.Axis(title='')

points = alt.Chart(dfs).mark_circle().encode(
    alt.X('GR', scale=xscale),
    alt.Y('ILD', scale=yscale),
    color='s_Facies',
)

top_hist = alt.Chart(dfs).mark_area(**area_args).encode(
    alt.X('GR:Q',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          bin=alt.Bin(maxbins=20, extent=xscale.domain),
          stack=None,
          axis=blank_axis,
         ),
    alt.Y('count()', stack=None, axis=blank_axis),
    alt.Color('s_Facies:N'),
).properties(height=60)

right_hist = alt.Chart(dfs).mark_area(**area_args).encode(
    alt.Y('ILD:Q',
          bin=alt.Bin(maxbins=20, extent=yscale.domain),
          stack=None,
          axis=blank_axis,
         ),
    alt.X('count()', stack=None, axis=blank_axis),
    alt.Color('s_Facies:N'),
).properties(width=60)

top_hist & (points | right_hist)

In [ ]:
brush = alt.selection_interval(encodings=['x'])
color = alt.Color('s_Facies:N')
xscale = alt.Scale(domain=(0, 350.0))
yscale = alt.Scale(domain=(1.9, 4.55))

area_args = {'opacity': .5, 'interpolate': 'step'}
blank_axis = alt.Axis(title='')

c1 = alt.Chart(dfs).mark_circle(opacity=.5).encode(
    alt.X('GR', type='quantitative'),
    alt.Y('RHOB', type='quantitative'),
    color=alt.condition(brush, color, alt.value('lightgray')),
).add_selection(brush)

c2 = alt.Chart(dfs).mark_circle(opacity=.5).encode(
    alt.X('GR', type='quantitative'),
    alt.Y('ILD', type='quantitative'),
    color=alt.condition(brush, color, alt.value('lightgray')),
).add_selection(brush)

top_hist = alt.Chart(dfs).mark_area(**area_args).encode(
    alt.X('GR:Q',
          bin=alt.Bin(maxbins=30, extent=xscale.domain),
          stack=None,
          axis=blank_axis,
         ),
    alt.Y('count()', stack=None, axis=blank_axis),
    alt.Color('s_Facies:N'),
).transform_filter(brush).properties(height=60)

top_hist & c1 & c2

## Holoviews

To install, activate `geocomp` and type:
- `pip install "holoviews[recommended]"`

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from holoviews.operation import gridmatrix

ds = hv.Dataset(dfs[['GR','RHOB','PE','ILD_log10','s_Facies']])

In [ ]:
%%opts Bivariate [bandwidth=0.5] (cmap=Cycle(values=['Blues', 'Reds', 'Oranges'])) Points (size=2 alpha=0.5)

grouped_by_facies = ds.groupby('s_Facies', container_type=hv.NdOverlay)
density_grid = gridmatrix(grouped_by_facies, diagonal_type=hv.Distribution, chart_type=hv.Bivariate)
point_grid = gridmatrix(grouped_by_facies, chart_type=hv.Points)

density_grid * point_grid

<hr />

<p style="color:gray">©2017 Agile Geoscience. Licensed CC-BY.</p>